# Sound2Sheet Training Pipeline

Complete training pipeline for audio-to-sheet-music transcription model.

## Step 1: Check GPU and Install Dependencies

In [18]:
import torch

# Check GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

Device: cpu


In [ ]:
# Clone repository
!git clone https://github.com/k-Dispersik/Sound2Sheet.git
%cd Sound2Sheet

In [ ]:
# Install system dependencies
!apt-get update -qq
!apt-get install -y -qq fluidsynth fluid-soundfont-gm

In [ ]:
# Install Python packages
!pip install -q torch torchaudio transformers librosa numpy pandas tqdm pyyaml \
    pretty_midi music21 matplotlib mido midi2audio accelerate scipy scikit-learn

print("Installation complete")

In [2]:
# Verify imports
from src.dataset import DatasetGenerator, MIDIGenerator
from src.core import AudioProcessor
from src.model import Sound2SheetModel, Trainer, create_dataloaders
from src.converter import NoteBuilder
from src.evaluation import Evaluator

print("All imports successful")

/home/volodymyr/VS code/My_projecs/Pythons/Sound2Sheet/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


All imports successful


## Step 2: Generate Dataset

Configure and generate synthetic training data.

In [5]:
# Dataset configuration
SAMPLES = 1000          # Total samples (train + val + test)
COMPLEXITY_DIST = "beginner:0.3,intermediate:0.5,advanced:0.2"  # Complexity distribution
EPOCHS = 50             # Training epochs
BATCH_SIZE = 16         # Batch size
LEARNING_RATE = 1e-4    # Learning rate
EXPERIMENT_NAME = "training_run_1"

print(f"Configuration:")
print(f"  Samples: {SAMPLES}")
print(f"  Complexity: {COMPLEXITY_DIST}")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Learning rate: {LEARNING_RATE}")

Configuration:
  Samples: 1000
  Complexity: beginner:0.3,intermediate:0.5,advanced:0.2
  Epochs: 50
  Batch size: 16
  Learning rate: 0.0001


In [6]:
# Generate dataset
!python -m src.dataset.cli generate \
    --samples {SAMPLES} \
    --complexity-dist {COMPLEXITY_DIST} \
    --name {EXPERIMENT_NAME} \
    --output-dir data/datasets/{EXPERIMENT_NAME}

Initializing dataset generator...
2025-11-06 20:50:52,412 - src.dataset.dataset_generator - INFO - Generating dataset: training_run_1 v1.0.0
2025-11-06 20:50:52,412 - src.dataset.dataset_generator - INFO - Total samples: 1000
2025-11-06 20:50:52,414 - src.dataset.dataset_generator - INFO - Split: train=700, val=150, test=150
2025-11-06 20:53:06,681 - src.dataset.dataset_generator - INFO - ✓ train: 700 samples, 488.6 MB audio + 378.3 KB MIDI
2025-11-06 20:53:06,681 - src.dataset.dataset_generator - INFO - ✓ train: 700 samples, 488.6 MB audio + 378.3 KB MIDI
2025-11-06 20:53:35,712 - src.dataset.dataset_generator - INFO - ✓ val: 150 samples, 101.5 MB audio + 76.0 KB MIDI
2025-11-06 20:53:35,712 - src.dataset.dataset_generator - INFO - ✓ val: 150 samples, 101.5 MB audio + 76.0 KB MIDI
2025-11-06 20:54:04,740 - src.dataset.dataset_generator - INFO - ✓ test: 150 samples, 106.5 MB audio + 79.8 KB MIDI
2025-11-06 20:54:04,757 - src.dataset.dataset_generator - INFO - Generated metadata files i

In [9]:
# Verify dataset (find the versioned directory)
import os
dataset_base = f'data/datasets/{EXPERIMENT_NAME}'
versioned_dirs = [d for d in os.listdir(dataset_base) if d.startswith(EXPERIMENT_NAME)]
if versioned_dirs:
    dataset_path = os.path.join(dataset_base, versioned_dirs[0])
    print(f"Found dataset: {dataset_path}")
    !python -m src.dataset.cli info {dataset_path}
else:
    print(f"No dataset found in {dataset_base}")

Found dataset: data/datasets/training_run_1/training_run_1_v1.0.0_20251106_205052
Dataset: training_run_1 v1.0.0
Generated: 2025-11-06T20:54:04.740127
Location: data/datasets/training_run_1/training_run_1_v1.0.0_20251106_205052

Total samples: 1000

Splits:
  train:  700 samples
  val  :  150 samples
  test :  150 samples

Configuration:
  Sample rate: 16000Hz
  Audio format: wav

Statistics:
  Complexity:
    beginner    : 266 ( 26.6%)
    intermediate: 535 ( 53.5%)
    advanced    : 199 ( 19.9%)
  Tempo: 60-180 BPM (avg: 120.0)
  Measures: 4-16 (avg: 9.9)
  Duration: 380.4 min total (avg: 22.8s per sample)
Dataset: training_run_1 v1.0.0
Generated: 2025-11-06T20:54:04.740127
Location: data/datasets/training_run_1/training_run_1_v1.0.0_20251106_205052

Total samples: 1000

Splits:
  train:  700 samples
  val  :  150 samples
  test :  150 samples

Configuration:
  Sample rate: 16000Hz
  Audio format: wav

Statistics:
  Complexity:
    beginner    : 266 ( 26.6%)
    intermediate: 535 ( 5

## Step 3: Configure Model

Set up model architecture and training parameters.

In [14]:
from src.model.config import ModelConfig, TrainingConfig, DataConfig
from pathlib import Path

# Use dataset_path from previous step
# dataset_path was already set in the verification step above

# Model configuration
model_config = ModelConfig(
    vocab_size=128,
    hidden_size=256,
    num_decoder_layers=6,
    num_attention_heads=8,
    dropout=0.1,
    max_sequence_length=512,
    device=device
)

# Training configuration
training_config = TrainingConfig(
    learning_rate=LEARNING_RATE,
    batch_size=BATCH_SIZE,
    num_epochs=EPOCHS,
    optimizer='adamw',
    scheduler='cosine',
    use_mixed_precision=True,
    max_grad_norm=1.0,
    gradient_accumulation_steps=4,
    checkpoint_dir='checkpoints',
    save_every_n_epochs=0,  # Only save best and final
    early_stopping_patience=10
)

# Data configuration
data_config = DataConfig(
    sample_rate=16000,
    n_mels=128,
    dataset_dir=Path(dataset_path)
)

print("Model configuration:")
print(f"  Hidden size: {model_config.hidden_size}")
print(f"  Decoder layers: {model_config.num_decoder_layers}")
print(f"  Attention heads: {model_config.num_attention_heads}")
print(f"\nTraining configuration:")
print(f"  Learning rate: {training_config.learning_rate}")
print(f"  Batch size: {training_config.batch_size}")
print(f"  Epochs: {training_config.num_epochs}")
print(f"  Mixed precision: {training_config.use_mixed_precision}")
print(f"  Gradient accumulation: {training_config.gradient_accumulation_steps}")
print(f"\nDataset path: {dataset_path}")

Model configuration:
  Hidden size: 256
  Decoder layers: 6
  Attention heads: 8

Training configuration:
  Learning rate: 0.0001
  Batch size: 16
  Epochs: 50
  Mixed precision: True
  Gradient accumulation: 4

Dataset path: data/datasets/training_run_1/training_run_1_v1.0.0_20251106_205052


## Step 4: Create Data Loaders

In [15]:
# Create dataloaders
train_loader, val_loader, test_loader = create_dataloaders(
    data_config, model_config, training_config
)

print(f"Dataset splits:")
print(f"  Train samples: {len(train_loader.dataset)}")
print(f"  Val samples: {len(val_loader.dataset)}")
print(f"  Test samples: {len(test_loader.dataset)}")

Dataset splits:
  Train samples: 700
  Val samples: 150
  Test samples: 150


## Step 5: Initialize Model

In [16]:
# Create model
model = Sound2SheetModel(model_config, freeze_encoder=True)

# Count parameters
params = model.count_parameters()
print(f"Model parameters:")
print(f"  Total: {params['total']:,}")
print(f"  Trainable: {params['trainable']:,}")
print(f"  Frozen: {params['frozen']:,}")

Model parameters:
  Total: 95,922,304
  Trainable: 9,735,040
  Frozen: 86,187,264


## Step 6: Train Model

Start training with progress tracking.

In [17]:
# Create trainer
trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    model_config=model_config,
    training_config=training_config
)

# Train
print("Starting training...")
trainer.train()
print("Training complete")

/home/volodymyr/VS code/My_projecs/Pythons/Sound2Sheet/src/model/trainer.py:97: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = GradScaler() if training_config.use_mixed_precision else None
/home/volodymyr/VS code/My_projecs/Pythons/Sound2Sheet/.venv/lib/python3.12/site-packages/torch/cuda/amp/grad_scaler.py:31: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  super().__init__(


Starting training...


Epoch 1:   0%|          | 0/44 [00:00<?, ?it/s]/home/volodymyr/VS code/My_projecs/Pythons/Sound2Sheet/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/volodymyr/VS code/My_projecs/Pythons/Sound2Sheet/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/volodymyr/VS code/My_projecs/Pythons/Sound2Sheet/src/model/trainer.py:248: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/home/volodymyr/VS code/My_projecs/Pythons/Sound2Sheet/.venv/lib/python3.12/site-packages/torch/amp/autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is no

KeyboardInterrupt: 

## Step 7: Visualize Training Results

In [ ]:
import json
import matplotlib.pyplot as plt
from pathlib import Path

# Load training history
history_file = Path('checkpoints/training_history.json')
if history_file.exists():
    with open(history_file, 'r') as f:
        history = json.load(f)
    
    # Plot training curves
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Loss
    axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2)
    axes[0].plot(history['val_loss'], label='Val Loss', linewidth=2)
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Training and Validation Loss')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # Accuracy
    axes[1].plot(history['val_accuracy'], label='Val Accuracy', color='green', linewidth=2)
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy')
    axes[1].set_title('Validation Accuracy')
    axes[1].legend()
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print best metrics
    best_epoch = history['val_loss'].index(min(history['val_loss']))
    print(f"Best results:")
    print(f"  Epoch: {best_epoch + 1}")
    print(f"  Val Loss: {history['val_loss'][best_epoch]:.4f}")
    print(f"  Val Accuracy: {history['val_accuracy'][best_epoch]:.4f}")
else:
    print("Training history not found")

## Step 8: Evaluate Model

Run evaluation on test set.

In [ ]:
from src.evaluation import EvaluationConfig

# Load best model
checkpoint = torch.load('checkpoints/best_model.pt', map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Evaluation configuration
eval_config = EvaluationConfig(
    onset_tolerance=0.05,
    offset_tolerance=0.05
)

# Create evaluator
evaluator = Evaluator(model, model_config, eval_config)

# Evaluate
print("Evaluating on test set...")
metrics = evaluator.evaluate(test_loader)

# Print results
print(f"\nEvaluation results:")
print(f"  Note Accuracy: {metrics['note_accuracy']:.2%}")
print(f"  Onset F1: {metrics['onset_f1']:.2%}")
print(f"  Offset F1: {metrics['offset_f1']:.2%}")
print(f"  Pitch Precision: {metrics['pitch_precision']:.2%}")
print(f"  Pitch Recall: {metrics['pitch_recall']:.2%}")
print(f"  Timing Deviation: {metrics['timing_deviation_ms']:.1f}ms")

## Step 9: Test Inference

Transcribe a sample from the test set.

In [ ]:
from src.model.config import InferenceConfig
from src.converter import NoteBuilder

# Get a test sample
test_sample = test_loader.dataset[0]
mel_spec = test_sample['mel_spec'].unsqueeze(0).to(device)

# Inference configuration
inference_config = InferenceConfig(
    strategy='greedy',
    max_length=512
)

# Generate predictions
print("Transcribing sample...")
with torch.no_grad():
    predictions = model.generate(mel_spec, inference_config)

print(f"Predicted {len(predictions)} notes")
print(f"Notes: {predictions[:20]}...")

# Convert to MIDI
note_builder = NoteBuilder()
note_sequence = note_builder.build_from_predictions(
    predictions,
    quantize=True,
    quantization_resolution=16
)

# Save MIDI
note_sequence.to_midi('transcribed_sample.mid')
print("\nSaved: transcribed_sample.mid")

## Step 10: Save Final Model

Save the trained model checkpoint.

In [ ]:
# Model is already saved in checkpoints/
# best_model.pt - best model by validation loss
# final_model.pt - final model after all epochs

print("Model files:")
print("  checkpoints/best_model.pt")
print("  checkpoints/final_model.pt")
print("  checkpoints/training_history.json")
print("\nDownload these files to use the model locally")

## Optional: Resume Training

If training was interrupted, resume from checkpoint.

In [ ]:
# Resume training example (uncomment to use)
"""
# Load checkpoint
checkpoint = torch.load('checkpoints/best_model.pt', map_location=device)

# Create new trainer with loaded model
trainer = Trainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    model_config=model_config,
    training_config=training_config,
    resume_from='checkpoints/best_model.pt'
)

# Continue training
trainer.train()
"""
print("Resume training code available (commented out)")

## Optional: Custom Configuration

Adjust hyperparameters for different experiments.

In [ ]:
# Example: Larger model
"""
model_config = ModelConfig(
    hidden_size=512,
    num_decoder_layers=8,
    num_attention_heads=16,
    dropout=0.1
)

# Example: More aggressive training
training_config = TrainingConfig(
    learning_rate=5e-5,
    batch_size=8,
    gradient_accumulation_steps=16,
    num_epochs=100
)
"""
print("Custom configuration examples available (commented out)")

## Training Complete

Model is trained and ready for use. Download checkpoint files to use locally.